# SPRINT 1 — Data Understanding & Cleaning

## Task 1: Load the dataset & print schema

In [ ]:
import pandas as pd
import os

# Load the dataset from Excel file
file_path = '../data/Cleaned_Viral_Social_Media_Trends.xlsx'
df = pd.read_excel(file_path)

print("=" * 80)
print("DATASET SCHEMA INFORMATION")
print("=" * 80)

# Print shape
print(f"\nDataset Shape: {df.shape}")
print(f"  - Rows: {df.shape[0]}")
print(f"  - Columns: {df.shape[1]}")

# Print info()
print("\n" + "=" * 80)
print("Dataset Info:")
print("=" * 80)
df.info()

# Print head()
print("\n" + "=" * 80)
print("First 5 Rows:")
print("=" * 80)
print(df.head())

## Task 2: Detect and remove duplicates

In [ ]:
# Task 2: Detect and remove duplicates
print("\n" + "=" * 80)
print("TASK 2: DETECT AND REMOVE DUPLICATES")
print("=" * 80)

# Check for duplicates before removal
duplicates_count = df.duplicated().sum()
print(f"\nNumber of duplicate rows (before removal): {duplicates_count}")

# Get duplicate rows by all columns
print(f"\nDuplicate rows by all columns: {df.duplicated().sum()}")

# Get duplicate rows by specific columns (if needed to check specific column duplicates)
print(f"\nDataset shape before removing duplicates: {df.shape}")

# Remove duplicates
df_cleaned = df.drop_duplicates()

print(f"Dataset shape after removing duplicates: {df_cleaned.shape}")

# Count of removed duplicates
removed_count = df.shape[0] - df_cleaned.shape[0]
print(f"\nTotal duplicates removed: {removed_count}")

# Update df to the cleaned version
df = df_cleaned

print("\n" + "=" * 80)
print(f"✓ Duplicates removed successfully. Dataset now has {df.shape[0]} unique rows.")
print("=" * 80)

In [ ]:
# Task 3: Handle missing values
print("\n" + "=" * 80)
print("TASK 3: HANDLE MISSING VALUES")
print("=" * 80)

# Generate missing value report
print("\nMissing Value Report:")
print("-" * 80)

missing_report = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum().values,
    'Missing_Percentage': (df.isnull().sum().values / len(df) * 100).round(2)
})

print(missing_report.to_string(index=False))

# Summary statistics
total_missing = df.isnull().sum().sum()
total_cells = df.shape[0] * df.shape[1]
missing_percentage = (total_missing / total_cells * 100).round(2)

print(f"\nTotal missing values: {total_missing}")
print(f"Total cells: {total_cells}")
print(f"Overall missing percentage: {missing_percentage}%")

# Handle missing values based on data type
print("\n" + "-" * 80)
print("Handling missing values:")
print("-" * 80)

df_filled = df.copy()

# For numeric columns: fill with median
numeric_cols = df.select_dtypes(include=['number']).columns
for col in numeric_cols:
    if df_filled[col].isnull().sum() > 0:
        median_val = df_filled[col].median()
        df_filled[col].fillna(median_val, inplace=True)
        print(f"  ✓ {col}: filled {df[col].isnull().sum()} missing values with median ({median_val:.2f})")

# For categorical columns: fill with mode
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if df_filled[col].isnull().sum() > 0:
        mode_val = df_filled[col].mode()[0]
        df_filled[col].fillna(mode_val, inplace=True)
        print(f"  ✓ {col}: filled {df[col].isnull().sum()} missing values with mode ('{mode_val}')")

# Verify missing values after handling
print("\n" + "-" * 80)
print("Verification after handling missing values:")
print("-" * 80)
remaining_missing = df_filled.isnull().sum().sum()
print(f"Total remaining missing values: {remaining_missing}")

if remaining_missing == 0:
    print("✓ All missing values have been handled successfully!")
else:
    print(f"⚠ Still {remaining_missing} missing values remaining")

# Update df to the cleaned version
df = df_filled

print("\n" + "=" * 80)
print(f"✓ Missing values handled. Dataset now has {df.shape[0]} rows and {df.shape[1]} columns.")
print("=" * 80)

## Task 3: Handle missing values

In [ ]:
# Task 4 (single cleaned_text column)
print("\n" + "=" * 80)
print("TASK 4 (single column): TEXT NORMALIZATION -> cleaned_text")
print("=" * 80)

import re

def normalize_text(text):
    """Normalize text according to requirements:
    - lowercase
    - remove URLs
    - remove punctuations
    - remove numbers
    - remove extra spaces
    """
    if pd.isna(text):
        return text
    text = str(text)
    # lowercase
    text = text.lower()
    # remove URLs
    text = re.sub(r'http\S+|www\.\S+|https\S+', '', text)
    # remove punctuation (keep alphanumeric and spaces)
    text = re.sub(r'[^\w\s]', ' ', text)
    # remove numbers
    text = re.sub(r'\d+', '', text)
    # replace underscores (from \w) and collapse spaces
    text = text.replace('_', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Choose a primary raw text column to create `cleaned_text` from
candidates = [col for col in df.columns 
              if df[col].dtype == 'object' 
              and '_normalized' not in col 
              and '_no_stopwords' not in col 
              and '_hashtags' not in col]

if candidates:
    primary_text_col = candidates[0]
    print(f"Using primary text column: {primary_text_col} -> creating 'cleaned_text'.")
    df['cleaned_text'] = df[primary_text_col].apply(normalize_text)
    print(f"\nDataset shape after adding 'cleaned_text': {df.shape}")
    print("\nSample cleaned_text (first 5 rows):")
    print(df['cleaned_text'].head().to_string(index=False))
else:
    print("No suitable text column found to create 'cleaned_text'.")

print("\n" + "=" * 80)

## Task 5: Remove stopwords

Input: `cleaned_text`
Output: `text_no_stopwords`

In [ ]:
# Task 5: Remove stopwords
print("\n" + "=" * 80)
print("TASK 5: REMOVE STOPWORDS")
print("=" * 80)

# Import nltk stopwords
import nltk
from nltk.corpus import stopwords

# Download stopwords if not already present
try:
    stopwords.words('english')
except LookupError:
    print("Downloading NLTK stopwords...")
    nltk.download('stopwords')

# Get English stopwords
stop_words = set(stopwords.words('english'))
print(f"\nLoaded {len(stop_words)} English stopwords")
print(f"Sample stopwords: {list(stop_words)[:10]}")

# Define function to remove stopwords
def remove_stopwords(text):
    """Remove stopwords from text."""
    if pd.isna(text) or text == '':
        return text
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Apply to cleaned_text column
if 'cleaned_text' in df.columns:
    print("\nApplying stopword removal to 'cleaned_text'...")
    df['text_no_stopwords'] = df['cleaned_text'].apply(remove_stopwords)
    print(f"✓ Created 'text_no_stopwords' column")
    print(f"Dataset shape: {df.shape}")
    
    # Show comparison
    print("\n" + "-" * 80)
    print("Sample comparison (cleaned_text vs text_no_stopwords):")
    print("-" * 80)
    
    sample_df = df[['cleaned_text', 'text_no_stopwords']].head(5)
    for idx, row in sample_df.iterrows():
        print(f"\nRow {idx}:")
        print(f"  cleaned_text: {row['cleaned_text'][:80]}...")
        print(f"  text_no_stopwords: {row['text_no_stopwords'][:80]}...")
    
    # Statistics
    print("\n" + "-" * 80)
    print("Statistics:")
    print("-" * 80)
    empty_count = (df['text_no_stopwords'] == '').sum()
    non_empty_count = len(df) - empty_count
    print(f"Non-empty text_no_stopwords: {non_empty_count} / {len(df)}")
    print(f"Empty text_no_stopwords: {empty_count} / {len(df)}")
    
else:
    print("⚠ 'cleaned_text' column not found. Please run Task 4 first.")

print("\n" + "=" * 80)
print("✓ Task 5 completed: Stopwords removed successfully!")
print("=" * 80)

## Task 6: Extract hashtags

Input: raw text
Output: `hashtags` column (list of hashtags)

In [ ]:
# Task 6: Extract hashtags
print("\n" + "=" * 80)
print("TASK 6: EXTRACT HASHTAGS")
print("=" * 80)

import re

# Define function to extract hashtags
def extract_hashtags(text):
    """Extract all hashtags from text."""
    if pd.isna(text):
        return []
    text = str(text)
    # Match hashtag pattern: # followed by word characters
    hashtags = re.findall(r'#\w+', text)
    return hashtags

# Find the primary text column (raw text, not processed)
candidates = [col for col in df.columns 
              if df[col].dtype == 'object' 
              and '_normalized' not in col 
              and '_no_stopwords' not in col 
              and 'cleaned' not in col]

if candidates:
    raw_text_col = candidates[0]
    print(f"Extracting hashtags from column: '{raw_text_col}'")
    df['hashtags'] = df[raw_text_col].apply(extract_hashtags)
    print(f"✓ Created 'hashtags' column")
    print(f"Dataset shape: {df.shape}")
    
    # Statistics
    print("\n" + "-" * 80)
    print("Hashtag Extraction Statistics:")
    print("-" * 80)
    
    total_rows = len(df)
    rows_with_hashtags = (df['hashtags'].apply(len) > 0).sum()
    rows_without_hashtags = total_rows - rows_with_hashtags
    total_hashtags = sum(len(h) for h in df['hashtags'])
    avg_hashtags_per_row = total_hashtags / total_rows if total_rows > 0 else 0
    
    print(f"Total rows: {total_rows}")
    print(f"Rows with hashtags: {rows_with_hashtags} ({rows_with_hashtags/total_rows*100:.2f}%)")
    print(f"Rows without hashtags: {rows_without_hashtags} ({rows_without_hashtags/total_rows*100:.2f}%)")
    print(f"Total hashtags extracted: {total_hashtags}")
    print(f"Average hashtags per row: {avg_hashtags_per_row:.2f}")
    
    # Show samples
    print("\n" + "-" * 80)
    print("Sample hashtags (first 10 rows with hashtags):")
    print("-" * 80)
    
    rows_with_tags = df[df['hashtags'].apply(len) > 0].head(10)
    for idx, (i, row) in enumerate(rows_with_tags.iterrows(), 1):
        hashtag_list = ', '.join(row['hashtags'][:5])  # Show max 5 hashtags
        if len(row['hashtags']) > 5:
            hashtag_list += f", ... (+{len(row['hashtags']) - 5} more)"
        print(f"{idx}. {hashtag_list}")
    
else:
    print("⚠ No suitable raw text column found to extract hashtags.")

print("\n" + "=" * 80)
print("✓ Task 6 completed: Hashtags extracted successfully!")
print("=" * 80)

## Task 7: Calculate sentiment scores

Input: clean text
Output: `polarity`, `subjectivity`

In [ ]:
# Task 7: Calculate sentiment scores
print("\n" + "=" * 80)
print("TASK 7: CALCULATE SENTIMENT SCORES (POLARITY & SUBJECTIVITY)")
print("=" * 80)

# Try to import TextBlob, install if missing
try:
    from textblob import TextBlob
except Exception:
    import sys, subprocess
    print("TextBlob not found. Installing textblob...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'textblob'])
    from textblob import TextBlob
    # Ensure punkt tokenizer available
    try:
        import nltk
        nltk.data.find('tokenizers/punkt')
    except Exception:
        import nltk
        print('Downloading NLTK punkt tokenizer...')
        nltk.download('punkt')

# Choose input column: prefer 'text_no_stopwords', then 'cleaned_text', then common raw text names
candidates = ['text_no_stopwords', 'cleaned_text', 'text', 'tweet', 'content', 'post', 'message', 'caption', 'body']
input_col = None
for c in candidates:
    if c in df.columns:
        input_col = c
        break

if input_col is None:
    raise ValueError('No suitable text column found for sentiment calculation. Please create `cleaned_text` or `text_no_stopwords` first.')

print(f"Using '{input_col}' as input for sentiment calculation")

# Function to compute polarity & subjectivity
def compute_sentiment(text):
    if pd.isna(text) or str(text).strip() == '':
        return (None, None)
    try:
        s = TextBlob(str(text))
        return (s.sentiment.polarity, s.sentiment.subjectivity)
    except Exception as e:
        return (None, None)

# Apply to the dataframe
sentiments = df[input_col].fillna('').apply(lambda t: compute_sentiment(t))

df['polarity'] = sentiments.apply(lambda x: x[0])
df['subjectivity'] = sentiments.apply(lambda x: x[1])

# Quick stats and sample
print(f"\nAdded columns: 'polarity' and 'subjectivity' to DataFrame. Dataset shape: {df.shape}")

print("\nSample sentiment scores (first 10 rows):")
print(df[[input_col, 'polarity', 'subjectivity']].head(10).to_string(index=False))

# Summary statistics
valid_polarity = df['polarity'].dropna()
valid_subjectivity = df['subjectivity'].dropna()
print("\n" + "-" * 80)
print("Sentiment summary:")
print("-" * 80)
if len(valid_polarity) > 0:
    print(f"Polarity: mean={valid_polarity.mean():.4f}, min={valid_polarity.min():.4f}, max={valid_polarity.max():.4f}")
else:
    print("No valid polarity values computed.")

if len(valid_subjectivity) > 0:
    print(f"Subjectivity: mean={valid_subjectivity.mean():.4f}, min={valid_subjectivity.min():.4f}, max={valid_subjectivity.max():.4f}")
else:
    print("No valid subjectivity values computed.")

print("\n" + "=" * 80)
print("✓ Task 7 completed: Sentiment scores calculated.")
print("=" * 80)

## Task 8: Export cleaned dataset

Input: processed DataFrame
Output: CSV → `processed/clean_data.csv`

In [ ]:
# Task 8: Export cleaned dataset
print("\n" + "=" * 80)
print("TASK 8: EXPORT CLEANED DATASET")
print("=" * 80)

import os

# Create output directory if it doesn't exist
output_dir = '../data/processed'
os.makedirs(output_dir, exist_ok=True)

# Define output file path
output_file = os.path.join(output_dir, 'clean_data.csv')

# Export to CSV
print(f"\nExporting cleaned DataFrame to: {output_file}")
df.to_csv(output_file, index=False, encoding='utf-8')

# Verify export
if os.path.exists(output_file):
    file_size = os.path.getsize(output_file) / (1024 * 1024)  # Convert to MB
    print(f"✓ Export successful!")
    print(f"  - File path: {output_file}")
    print(f"  - File size: {file_size:.2f} MB")
    print(f"  - Rows: {len(df)}")
    print(f"  - Columns: {len(df.columns)}")
    print(f"\n  Columns in cleaned dataset:")
    for i, col in enumerate(df.columns, 1):
        print(f"    {i}. {col}")
else:
    print(f"⚠ Export failed. File not found at {output_file}")

print("\n" + "=" * 80)
print("✓ Task 8 completed: Cleaned dataset exported successfully!")
print("=" * 80)

# SPRINT 2 — Sentiment Drivers + Hashtag Intelligence

## Task 9: Generate unigrams/bigrams/trigrams

Output: frequency tables for positive & negative sentiments

In [ ]:
# Task 9: Generate unigrams/bigrams/trigrams
print("\n" + "=" * 80)
print("TASK 9: GENERATE N-GRAMS (UNIGRAMS, BIGRAMS, TRIGRAMS)")
print("=" * 80)

from collections import Counter
from nltk.util import ngrams

# Define sentiment thresholds
# Positive: polarity > 0.1, Negative: polarity < -0.1, Neutral: in between
positive_threshold = 0.1
negative_threshold = -0.1

# Categorize rows by sentiment
df['sentiment_category'] = df['polarity'].apply(
    lambda x: 'positive' if x > positive_threshold 
              else ('negative' if x < negative_threshold else 'neutral')
)

print(f"\nSentiment distribution:")
print(f"  Positive (polarity > {positive_threshold}): {(df['sentiment_category'] == 'positive').sum()}")
print(f"  Negative (polarity < {negative_threshold}): {(df['sentiment_category'] == 'negative').sum()}")
print(f"  Neutral: {(df['sentiment_category'] == 'neutral').sum()}")

# Function to generate n-grams
def generate_ngrams(text, n):
    """Generate n-grams from text."""
    if pd.isna(text) or str(text).strip() == '':
        return []
    words = str(text).split()
    return list(ngrams(words, n))

# Generate n-grams for positive and negative sentiments
print("\n" + "-" * 80)
print("Generating N-grams for Positive Sentiment:")
print("-" * 80)

positive_texts = df[df['sentiment_category'] == 'positive']['text_no_stopwords']
positive_unigrams = Counter()
positive_bigrams = Counter()
positive_trigrams = Counter()

for text in positive_texts:
    positive_unigrams.update(generate_ngrams(text, 1))
    positive_bigrams.update(generate_ngrams(text, 2))
    positive_trigrams.update(generate_ngrams(text, 3))

print(f"Top 10 Unigrams (Positive):")
for gram, freq in positive_unigrams.most_common(10):
    print(f"  {gram[0]}: {freq}")

print(f"\nTop 10 Bigrams (Positive):")
for gram, freq in positive_bigrams.most_common(10):
    print(f"  {' '.join(gram)}: {freq}")

print(f"\nTop 10 Trigrams (Positive):")
for gram, freq in positive_trigrams.most_common(10):
    print(f"  {' '.join(gram)}: {freq}")

# Generate n-grams for negative sentiment
print("\n" + "-" * 80)
print("Generating N-grams for Negative Sentiment:")
print("-" * 80)

negative_texts = df[df['sentiment_category'] == 'negative']['text_no_stopwords']
negative_unigrams = Counter()
negative_bigrams = Counter()
negative_trigrams = Counter()

for text in negative_texts:
    negative_unigrams.update(generate_ngrams(text, 1))
    negative_bigrams.update(generate_ngrams(text, 2))
    negative_trigrams.update(generate_ngrams(text, 3))

print(f"Top 10 Unigrams (Negative):")
for gram, freq in negative_unigrams.most_common(10):
    print(f"  {gram[0]}: {freq}")

print(f"\nTop 10 Bigrams (Negative):")
for gram, freq in negative_bigrams.most_common(10):
    print(f"  {' '.join(gram)}: {freq}")

print(f"\nTop 10 Trigrams (Negative):")
for gram, freq in negative_trigrams.most_common(10):
    print(f"  {' '.join(gram)}: {freq}")

# Create frequency DataFrames for export
pos_unigram_df = pd.DataFrame(
    [(gram[0], freq) for gram, freq in positive_unigrams.most_common(50)],
    columns=['unigram', 'frequency']
)
pos_bigram_df = pd.DataFrame(
    [(' '.join(gram), freq) for gram, freq in positive_bigrams.most_common(50)],
    columns=['bigram', 'frequency']
)
pos_trigram_df = pd.DataFrame(
    [(' '.join(gram), freq) for gram, freq in positive_trigrams.most_common(50)],
    columns=['trigram', 'frequency']
)

neg_unigram_df = pd.DataFrame(
    [(gram[0], freq) for gram, freq in negative_unigrams.most_common(50)],
    columns=['unigram', 'frequency']
)
neg_bigram_df = pd.DataFrame(
    [(' '.join(gram), freq) for gram, freq in negative_bigrams.most_common(50)],
    columns=['bigram', 'frequency']
)
neg_trigram_df = pd.DataFrame(
    [(' '.join(gram), freq) for gram, freq in negative_trigrams.most_common(50)],
    columns=['trigram', 'frequency']
)

print("\n" + "-" * 80)
print("Summary:")
print("-" * 80)
print(f"Positive unigrams: {len(positive_unigrams)}")
print(f"Positive bigrams: {len(positive_bigrams)}")
print(f"Positive trigrams: {len(positive_trigrams)}")
print(f"Negative unigrams: {len(negative_unigrams)}")
print(f"Negative bigrams: {len(negative_bigrams)}")
print(f"Negative trigrams: {len(negative_trigrams)}")

print("\n" + "=" * 80)
print("✓ Task 9 completed: N-grams generated and analyzed!")
print("=" * 80)

## Task 10: Create TF-IDF vectors (positive vs negative groups)

Output: top TF-IDF keywords per sentiment

In [ ]:
# Task 10: Create TF-IDF vectors (positive vs negative groups)
print("\n" + "=" * 80)
print("TASK 10: CREATE TF-IDF VECTORS (POSITIVE VS NEGATIVE GROUPS)")
print("=" * 80)

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Select sentiment column
sentiment_col = 'sentiment_category'

# Find best text column to use (prefer cleaned_text, then raw text columns)
text_col = None
candidates = ['cleaned_text', 'text', 'tweet', 'content', 'post', 'message', 'caption', 'body']
for col in candidates:
    if col in df.columns:
        text_col = col
        break

if text_col is None:
    raise ValueError('No suitable text column found for TF-IDF calculation.')

print(f"Using '{text_col}' column for TF-IDF analysis")

# Prepare positive and negative groups
positive_texts = df[df[sentiment_col] == 'positive'][text_col].fillna('').tolist()
negative_texts = df[df[sentiment_col] == 'negative'][text_col].fillna('').tolist()

print(f"Positive texts: {len(positive_texts)}, Negative texts: {len(negative_texts)}")

# Combine for vectorizer fit
all_texts = positive_texts + negative_texts
labels = (['positive'] * len(positive_texts)) + (['negative'] * len(negative_texts))

# Fit TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english', min_df=2, max_df=0.8)
X = vectorizer.fit_transform(all_texts)
feature_names = np.array(vectorizer.get_feature_names_out())

# Split back to positive/negative
X_pos = X[:len(positive_texts)]
X_neg = X[len(positive_texts):]

# Compute mean TF-IDF per word for each group
mean_tfidf_pos = np.asarray(X_pos.mean(axis=0)).flatten()
mean_tfidf_neg = np.asarray(X_neg.mean(axis=0)).flatten()

# Get top keywords per group
top_n = 20
pos_top_idx = mean_tfidf_pos.argsort()[::-1][:top_n]
neg_top_idx = mean_tfidf_neg.argsort()[::-1][:top_n]

print("\n" + "-" * 80)
print("Top TF-IDF keywords for POSITIVE sentiment:")
print("-" * 80)
for i in pos_top_idx:
    print(f"  {feature_names[i]}: {mean_tfidf_pos[i]:.4f}")

print("\n" + "-" * 80)
print("Top TF-IDF keywords for NEGATIVE sentiment:")
print("-" * 80)
for i in neg_top_idx:
    print(f"  {feature_names[i]}: {mean_tfidf_neg[i]:.4f}")

print("\n" + "=" * 80)
print("✓ Task 10 completed: Top TF-IDF keywords per sentiment generated!")
print("=" * 80)

## Task 11: Emoji extraction + sentiment mapping

Output: table of emoji → sentiment score

In [ ]:
# Task 11: Emoji extraction + sentiment mapping
print("\n" + "=" * 80)
print("TASK 11: EMOJI EXTRACTION + SENTIMENT MAPPING")
print("=" * 80)

import re

# Try to import emoji library, install if missing
try:
    import emoji
except Exception:
    import sys, subprocess
    print("emoji library not found. Installing emoji...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'emoji'])
    import emoji

# Find raw text column (original unprocessed text with emojis)
raw_text_col = None
candidates = [col for col in df.columns 
              if df[col].dtype == 'object' 
              and 'cleaned' not in col 
              and '_no_stopwords' not in col 
              and 'hashtags' not in col]

if candidates:
    raw_text_col = candidates[0]
    print(f"Using '{raw_text_col}' column for emoji extraction")
else:
    raise ValueError('No suitable raw text column found for emoji extraction.')

# Function to extract emojis from text
def extract_emojis(text):
    """Extract all emojis from text."""
    if pd.isna(text):
        return []
    text = str(text)
    # Use emoji library to find emojis
    emoji_list = [char for char in text if char in emoji.EMOJI_DATA]
    return emoji_list

# Extract emojis from raw text
print("\nExtracting emojis from text...")
df['emojis'] = df[raw_text_col].apply(extract_emojis)

# Flatten all emojis and create sentiment mapping
emoji_sentiment_map = {}
emoji_counts = {}

for idx, row in df.iterrows():
    emojis_in_text = row['emojis']
    polarity = row['polarity']
    
    for em in emojis_in_text:
        if em not in emoji_sentiment_map:
            emoji_sentiment_map[em] = []
            emoji_counts[em] = 0
        emoji_sentiment_map[em].append(polarity)
        emoji_counts[em] += 1

# Calculate average sentiment for each emoji
emoji_sentiment_summary = []
for em, sentiments in emoji_sentiment_map.items():
    avg_sentiment = np.mean(sentiments)
    std_sentiment = np.std(sentiments)
    count = len(sentiments)
    
    # Get emoji description
    try:
        emoji_desc = emoji.demojize(em)
    except:
        emoji_desc = "unknown"
    
    emoji_sentiment_summary.append({
        'emoji': em,
        'description': emoji_desc,
        'avg_polarity': avg_sentiment,
        'std_polarity': std_sentiment,
        'count': count
    })

# Create DataFrame and sort by count
emoji_df = pd.DataFrame(emoji_sentiment_summary)
emoji_df = emoji_df.sort_values('count', ascending=False)

print(f"\nTotal unique emojis found: {len(emoji_df)}")
print(f"Total emoji occurrences: {emoji_df['count'].sum()}")

# Display top emojis by frequency
print("\n" + "-" * 80)
print("Top 20 Emojis by Frequency + Sentiment Score:")
print("-" * 80)

if len(emoji_df) > 0:
    display_df = emoji_df.head(20)[['emoji', 'description', 'avg_polarity', 'count']]
    for idx, row in display_df.iterrows():
        emoji_char = row['emoji']
        desc = row['description']
        avg_pol = row['avg_polarity']
        count = row['count']
        sentiment_label = "Positive" if avg_pol > 0.1 else ("Negative" if avg_pol < -0.1 else "Neutral")
        print(f"{emoji_char} {desc}: avg_polarity={avg_pol:.4f} ({sentiment_label}), count={count}")
else:
    print("No emojis found in the dataset.")

# Statistics
print("\n" + "-" * 80)
print("Emoji Sentiment Statistics:")
print("-" * 80)
if len(emoji_df) > 0:
    print(f"Average polarity of emojis: {emoji_df['avg_polarity'].mean():.4f}")
    print(f"Most positive emoji avg_polarity: {emoji_df['avg_polarity'].max():.4f}")
    print(f"Most negative emoji avg_polarity: {emoji_df['avg_polarity'].min():.4f}")
    
    # Categorize emojis
    positive_emojis = len(emoji_df[emoji_df['avg_polarity'] > 0.1])
    negative_emojis = len(emoji_df[emoji_df['avg_polarity'] < -0.1])
    neutral_emojis = len(emoji_df[(emoji_df['avg_polarity'] >= -0.1) & (emoji_df['avg_polarity'] <= 0.1)])
    
    print(f"Positive emojis: {positive_emojis}")
    print(f"Negative emojis: {negative_emojis}")
    print(f"Neutral emojis: {neutral_emojis}")
else:
    print("No emojis found.")

print("\n" + "=" * 80)
print("✓ Task 11 completed: Emojis extracted and sentiment mapped!")
print("=" * 80)

## Task 12: Hashtag frequency analysis

Output: table of hashtags → count

In [ ]:
# Task 12: Hashtag frequency analysis
print("\n" + "=" * 80)
print("TASK 12: HASHTAG FREQUENCY ANALYSIS")
print("=" * 80)

from collections import Counter

# Extract and flatten all hashtags from the hashtags column
all_hashtags = []
for hashtag_list in df['hashtags']:
    if isinstance(hashtag_list, list):
        all_hashtags.extend(hashtag_list)

# Count hashtag frequencies
hashtag_counter = Counter(all_hashtags)

# Create summary DataFrame
hashtag_freq_df = pd.DataFrame(
    list(hashtag_counter.most_common()),
    columns=['hashtag', 'count']
)

# Add percentage column
total_hashtags = hashtag_freq_df['count'].sum()
hashtag_freq_df['percentage'] = (hashtag_freq_df['count'] / total_hashtags * 100).round(2)

print(f"\nTotal unique hashtags: {len(hashtag_freq_df)}")
print(f"Total hashtag occurrences: {total_hashtags}")

# Display top 30 hashtags
print("\n" + "-" * 80)
print("Top 30 Hashtags by Frequency:")
print("-" * 80)

top_30 = hashtag_freq_df.head(30)
for idx, row in top_30.iterrows():
    hashtag = row['hashtag']
    count = row['count']
    percentage = row['percentage']
    print(f"{hashtag}: {count} ({percentage}%)")

# Statistics
print("\n" + "-" * 80)
print("Hashtag Frequency Statistics:")
print("-" * 80)
print(f"Most frequent hashtag: {hashtag_freq_df.iloc[0]['hashtag']} ({hashtag_freq_df.iloc[0]['count']} occurrences)")
print(f"Average hashtag frequency: {hashtag_freq_df['count'].mean():.2f}")
print(f"Median hashtag frequency: {hashtag_freq_df['count'].median():.2f}")
print(f"Max frequency: {hashtag_freq_df['count'].max()}")
print(f"Min frequency: {hashtag_freq_df['count'].min()}")

# Trending analysis: hashtags appearing more than once
trending_threshold = 2
trending_hashtags = hashtag_freq_df[hashtag_freq_df['count'] >= trending_threshold]
print(f"\nHashtags with 2+ occurrences (trending): {len(trending_hashtags)}")
print(f"Hashtags appearing only once: {len(hashtag_freq_df[hashtag_freq_df['count'] == 1])}")

# Top hashtags contribution
top_10_contribution = top_30['count'].sum() / total_hashtags * 100
print(f"\nTop 10 hashtags contribute: {top_10_contribution:.2f}% of all hashtag occurrences")

print("\n" + "=" * 80)
print("✓ Task 12 completed: Hashtag frequency analysis completed!")
print("=" * 80)

## Task 13: Hashtag sentiment analysis

Output: `hashtag` → average sentiment table (avg_polarity, std_polarity, count, frequency, percentage)

In [ ]:
# Task 13: Hashtag sentiment analysis
print("\n" + "=" * 80)
print("TASK 13: HASHTAG SENTIMENT ANALYSIS")
print("=" * 80)

import os

# Check that required columns exist
if 'hashtags' not in df.columns:
    raise ValueError("Column 'hashtags' not found. Run Task 6 before Task 13.")
if 'polarity' not in df.columns:
    raise ValueError("Column 'polarity' not found. Run Task 7 before Task 13.")

# Explode hashtags to associate each hashtag with the row's polarity
expanded = df[['hashtags', 'polarity']].explode('hashtags')
# Drop rows with no hashtag (NaN after explode)
expanded = expanded.dropna(subset=['hashtags'])

# Normalize hashtags to consistent case (optional)
expanded['hashtag_norm'] = expanded['hashtags'].astype(str).str.lower()

# Compute aggregate sentiment per hashtag
hashtag_sentiment = expanded.groupby('hashtag_norm')['polarity'].agg(
    avg_polarity='mean', std_polarity='std', count='count'
).reset_index()

# If Task 12 produced `hashtag_freq_df` we can merge percentage; otherwise compute frequency from this table
if 'hashtag_freq_df' in globals():
    # Ensure hashtag column names align (hashtag_freq_df has 'hashtag')
    hashtag_sentiment = hashtag_sentiment.merge(
        hashtag_freq_df.rename(columns={'hashtag':'hashtag_norm'}),
        on='hashtag_norm', how='left'
    )
else:
    total_hashtags = hashtag_sentiment['count'].sum()
    hashtag_sentiment['percentage'] = (hashtag_sentiment['count'] / total_hashtags * 100).round(2)

# Sort by count descending
hashtag_sentiment = hashtag_sentiment.sort_values('count', ascending=False)

# Create output directory and save CSV
out_dir = '../data/processed'
os.makedirs(out_dir, exist_ok=True)
out_file = os.path.join(out_dir, 'hashtag_sentiment.csv')
hashtag_sentiment.to_csv(out_file, index=False, encoding='utf-8')

print(f"\nSaved hashtag sentiment table to: {out_file}")
print(f"Total hashtags in table: {len(hashtag_sentiment)}")

# Print top 30 hashtags with sentiment
print("\n" + "-" * 80)
print("Top 30 hashtags with average polarity and counts:")
print("-" * 80)
print(hashtag_sentiment.head(30).to_string(index=False))

print("\n" + "=" * 80)
print("✓ Task 13 completed: Hashtag sentiment table created and exported.")
print("=" * 80)

In [ ]:
# Task 15: Filter only negative sentiment records
print('\n' + '=' * 80)
print('TASK 15: FILTER NEGATIVE SENTIMENT RECORDS -> negative_df')
print('=' * 80)

import os

# Ensure polarity column exists
if 'polarity' not in df.columns:
    raise ValueError("Column 'polarity' not found. Run Task 7 to compute sentiment polarity before Task 15.")

# Ensure sentiment_category exists; compute if missing using same thresholds as Task 9
positive_threshold = 0.1
negative_threshold = -0.1
if 'sentiment_category' not in df.columns:
    print("'sentiment_category' not found — computing from 'polarity' using thresholds.")
    df['sentiment_category'] = df['polarity'].apply(
        lambda x: 'positive' if x > positive_threshold else ('negative' if x < negative_threshold else 'neutral')
    )

# Filter negatives
negative_df = df[df['sentiment_category'] == 'negative'].copy()

print(f"Negative records: {len(negative_df)} rows")
if len(negative_df) > 0:
    print('\nSample negative rows (first 5):')
    print(negative_df.head(5).to_string(index=False))
else:
    print('No negative records found.')

# Optionally save to CSV for downstream use
out_dir = '../data/processed'
os.makedirs(out_dir, exist_ok=True)
neg_path = os.path.join(out_dir, 'negative_df.csv')
negative_df.to_csv(neg_path, index=False, encoding='utf-8')
print(f"\nSaved negative_df to: {neg_path}")

print('\n' + '=' * 80)
print('✓ Task 15 completed: `negative_df` created and exported.')
print('=' * 80)

In [ ]:
# Task 16: Extract negative n-grams (bigrams & trigrams)
print('\n' + '=' * 80)
print('TASK 16: NEGATIVE BIGRAM & TRIGRAM FREQUENCY')
print('=' * 80)

from collections import Counter
from nltk.util import ngrams
import os
import pandas as pd

# Ensure we have a negative dataframe (negative_df) or compute it
if 'negative_df' not in globals():
    print('`negative_df` not found in memory — computing from `df`.')
    if 'polarity' not in df.columns:
        raise ValueError("Column 'polarity' not found. Run sentiment Task 7 first.")
    # use same thresholds as Task 9
    pos_thr = 0.1
    neg_thr = -0.1
    if 'sentiment_category' not in df.columns:
        df['sentiment_category'] = df['polarity'].apply(lambda x: 'positive' if x>pos_thr else ('negative' if x<neg_thr else 'neutral'))
    negative_df = df[df['sentiment_category'] == 'negative'].copy()

print(f'Negative rows to process: {len(negative_df)}')

# Pick text column preference (prefer tokenized/no-stopwords)
candidates = ['text_no_stopwords','cleaned_text','text','tweet','content','post','message','caption','body']
text_col = None
for c in candidates:
    if c in negative_df.columns:
        text_col = c
        break

if text_col is None:
    print('⚠ No suitable text column found in negative_df. Skipping n-gram extraction.')
else:
    print(f'Using `{text_col}` for n-gram extraction')
    texts = negative_df[text_col].fillna('').astype(str).tolist()
    bigram_counter = Counter()
    trigram_counter = Counter()

    for t in texts:
        tokens = t.split()
        if len(tokens) < 2:
            continue
        bigrams = ngrams(tokens, 2)
        trigrams = ngrams(tokens, 3) if len(tokens) >= 3 else []
        bigram_counter.update([' '.join(g) for g in bigrams])
        trigram_counter.update([' '.join(g) for g in trigrams])

    # Create DataFrames and export top N
    top_n = 200
    neg_bigram_df = pd.DataFrame(bigram_counter.most_common(top_n), columns=['bigram','frequency'])
    neg_trigram_df = pd.DataFrame(trigram_counter.most_common(top_n), columns=['trigram','frequency'])

    out_dir = '../data/processed'
    os.makedirs(out_dir, exist_ok=True)
    bigram_path = os.path.join(out_dir, 'negative_bigrams.csv')
    trigram_path = os.path.join(out_dir, 'negative_trigrams.csv')
    neg_bigram_df.to_csv(bigram_path, index=False, encoding='utf-8')
    neg_trigram_df.to_csv(trigram_path, index=False, encoding='utf-8')

    print('
Top 30 Negative Bigrams:')
    for gram, freq in neg_bigram_df.head(30).values:
        print(f'  {gram}: {freq}')

    print('
Top 30 Negative Trigrams:')
    for gram, freq in neg_trigram_df.head(30).values:
        print(f'  {gram}: {freq}')

    print(f'\nSaved negative bigrams to: {bigram_path}')
    print(f'Saved negative trigrams to: {trigram_path}')

print('\n' + '=' * 80)
print('✓ Task 16 completed: negative n-gram frequency tables created and exported.')
print('=' * 80)

In [ ]:
# Task 17: Topic modeling (LDA default, KMeans optional) on negative posts
print('\n' + '=' * 80)
print('TASK 17: TOPIC MODELING ON NEGATIVE POSTS')
print('=' * 80)

import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans

# Parameters - adjust if needed
n_topics = 8
top_n_words = 12
method = 'lda'  # choices: 'lda' or 'kmeans'
out_dir = '../data/processed'
os.makedirs(out_dir, exist_ok=True)

# Ensure negative_df exists (compute if necessary)
if 'negative_df' not in globals():
    print('`negative_df` not in memory — deriving from `df`')
    if 'polarity' not in df.columns:
        raise ValueError('Column 
 not found. Run Task 7 first.')
    pos_thr = 0.1
    neg_thr = -0.1
    if 'sentiment_category' not in df.columns:
        df['sentiment_category'] = df['polarity'].apply(lambda x: 'positive' if x>pos_thr else ('negative' if x<neg_thr else 'neutral'))
    negative_df = df[df['sentiment_category'] == 'negative'].copy()

print(f'Negative documents: {len(negative_df)}')
if len(negative_df) == 0:
    print('No negative documents to model. Skipping Task 17.')
else:
    # Choose text column
    candidates = ['text_no_stopwords','cleaned_text','text','tweet','content','post','message','caption','body']
    text_col = next((c for c in candidates if c in negative_df.columns), None)
    if text_col is None:
        raise ValueError('No suitable text column found for topic modeling.')
    print(f'Using `{text_col}` for topic modeling')
    docs = negative_df[text_col].fillna('').astype(str).tolist()

    try:
        if method == 'lda':
            print('Building document-term matrix (CountVectorizer)')
            vectorizer = CountVectorizer(max_df=0.95, min_df=3, max_features=5000, stop_words='english')
            dtm = vectorizer.fit_transform(docs)
            print('Fitting LDA model...')
            lda = LatentDirichletAllocation(n_components=n_topics, random_state=42, learning_method='batch', max_iter=20)
            lda.fit(dtm)
            # Extract topic keywords
            feature_names = vectorizer.get_feature_names_out()
            topics = []
            for topic_idx, topic in enumerate(lda.components_):
                top_features_ind = topic.argsort()[::-1][:top_n_words]
                top_features = [feature_names[i] for i in top_features_ind]
                topics.append({'topic': topic_idx, 'keywords': ' '.join(top_features)})
            topics_df = pd.DataFrame(topics)
            # Assign dominant topic to each document
            doc_topic_dist = lda.transform(dtm)
            doc_topics = doc_topic_dist.argmax(axis=1)
            negative_df['topic'] = ['topic_' + str(t) for t in doc_topics]
        else:
            # KMeans on TF-IDF
            print('Computing TF-IDF and running KMeans...')
            tfv = TfidfVectorizer(max_df=0.95, min_df=3, max_features=5000, stop_words='english')
            X = tfv.fit_transform(docs)
            kmeans = KMeans(n_clusters=n_topics, random_state=42, n_init=10)
            kmeans.fit(X)
            labels = kmeans.labels_
            negative_df['topic'] = ['topic_' + str(t) for t in labels]
            # extract top terms per cluster by centroid
            try:
                terms = tfv.get_feature_names_out()
                order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
                topics = []
                for i in range(n_topics):
                    top_terms = [terms[ind] for ind in order_centroids[i, :top_n_words]]
                    topics.append({'topic': i, 'keywords': ' '.join(top_terms)})
                topics_df = pd.DataFrame(topics)
            except Exception:
                topics_df = pd.DataFrame([])
        # Save outputs
        topics_path = os.path.join(out_dir, 'negative_topics_keywords.csv')
        topics_df.to_csv(topics_path, index=False, encoding='utf-8')
        neg_topics_path = os.path.join(out_dir, 'negative_df_topics.csv')
        negative_df.to_csv(neg_topics_path, index=False, encoding='utf-8')
        print(f'✓ Saved topic keywords to: {topics_path}')
        print(f'✓ Saved negative_df with topic labels to: {neg_topics_path}')
        print('
Top topics (keywords):')
        print(topics_df.head(20).to_string(index=False))
    except Exception as e:
        print('⚠ Topic modeling failed:', e)

print('
' + '=' * 80)
print('✓ Task 17 completed: Topic modeling executed and outputs saved (if successful).')
print('=' * 80)

In [ ]:
# Task 18: Summarize topic clusters
print('\n' + '=' * 80)
print('TASK 18: SUMMARIZE TOPIC CLUSTERS')
print('=' * 80)
import os
import pandas as pd

out_dir = '../data/processed'
topics_path = os.path.join(out_dir, 'negative_topics_keywords.csv')

# Load topics_df from memory or CSV
if 'topics_df' not in globals():
    if os.path.exists(topics_path):
        topics_df = pd.read_csv(topics_path)
        print(f\
    else:
        raise ValueError(\
17
)

# Ensure keywords column is available
if 'keywords' not in topics_df.columns:
    raise ValueError(\
)

# Build summary with cluster_name and key_terms
def format_cluster_name(t):
    try:
        return f\

    except Exception:
        return str(t)

summary_df = pd.DataFrame({
    'cluster_name': topics_df['topic'].apply(format_cluster_name) if 'topic' in topics_df.columns else ['cluster_' + str(i) for i in topics_df.index],
    'key_terms': topics_df['keywords']
})

# Save summary
summary_path = os.path.join(out_dir, 'negative_topics_summary.csv')
summary_df.to_csv(summary_path, index=False, encoding='utf-8')
print(f\
print('\nTop clusters (name + key terms):')
print(summary_df.head(50).to_string(index=False))

print('\n' + '=' * 80)
print('✓ Task 18 completed: Topic clusters summarized and exported.')
print('=' * 80)

In [ ]:
# Task 19: Identify toxic keywords
print('\n' + '=' * 80)
print('TASK 19: IDENTIFY TOXIC KEYWORDS')
print('=' * 80)
import os
import re
import pandas as pd
from collections import Counter

out_dir = '../data/processed'
os.makedirs(out_dir, exist_ok=True)
# Prefer a supplied lexicon in processed/; fall back to a small default list
lex_csv = os.path.join(out_dir, 'toxicity_lexicon.csv')
lex_txt = os.path.join(out_dir, 'toxicity_lexicon.txt')
toxicity_lexicon = None
if os.path.exists(lex_csv):
    try:
        toxicity_lexicon = pd.read_csv(lex_csv, header=None).iloc[:,0].astype(str).str.lower().str.strip().dropna().unique().tolist()
        print(f'Loaded toxicity lexicon from: {lex_csv}')
    except Exception as e:
        print('Failed to read lexicon CSV:', e)
elif os.path.exists(lex_txt):
    try:
        with open(lex_txt, 'r', encoding='utf-8') as f:
            toxicity_lexicon = [l.strip().lower() for l in f if l.strip()]
        print(f'Loaded toxicity lexicon from: {lex_txt}')
    except Exception as e:
        print('Failed to read lexicon TXT:', e)
else:
    # Minimal example lexicon; replace with a fuller lexicon file for production
    toxicity_lexicon = ['insult', 'hate', 'scam', 'fraud', 'abuse', 'spam', 'stupid', 'idiot', 'terrible', 'awful']
    print('No lexicon file found — using built-in minimal lexicon (recommend providing a larger list at data/processed/toxicity_lexicon.csv)')

# Choose text column (prefer tokenized/no-stopwords)
candidates = ['text_no_stopwords','cleaned_text','text','tweet','content','post','message','caption','body']
text_col = next((c for c in candidates if c in globals().get('df', {}) and c in df.columns), None)
if text_col is None:
    # fall back to checking df columns directly
    text_col = next((c for c in candidates if c in df.columns), None)
if text_col is None:
    raise ValueError('No suitable text column found for toxicity detection. Create `cleaned_text` or `text_no_stopwords` first.')
print(f'Using 
 as source for toxicity counting')

# Prepare pattern list (word-boundary safe)
tox_patterns = [re.compile(r'\b' + re.escape(w) + r'\b', flags=re.IGNORECASE) for w in toxicity_lexicon]

# Count occurrences across documents
counter = Counter()
rows_with_toxic = 0
for text in df[text_col].fillna('').astype(str):
    text_lower = text.lower()
    found_any = False
    for w, pat in zip(toxicity_lexicon, tox_patterns):
        matches = len(pat.findall(text_lower))
        if matches > 0:
            counter[w] += matches
            found_any = True
    if found_any:
        rows_with_toxic += 1

# Build DataFrame of frequencies
tox_df = pd.DataFrame([(w, counter.get(w, 0)) for w in toxicity_lexicon], columns=['toxic_word','frequency'])
tox_df = tox_df.sort_values('frequency', ascending=False).reset_index(drop=True)

# Save output
out_file = os.path.join(out_dir, 'toxic_words_frequency.csv')
tox_df.to_csv(out_file, index=False, encoding='utf-8')

print(f'
Saved toxic-word frequency table to: {out_file}')
print(f'Total rows scanned: {len(df)}; rows containing >=1 toxic word: {rows_with_toxic}')
print('
Top toxic words by frequency:')
print(tox_df[tox_df['frequency']>0].head(50).to_string(index=False))

print('\n' + '=' * 80)
print('✓ Task 19 completed: Toxic keywords identified and frequencies exported.')
print('=' * 80)

In [ ]:
# Task 21: Create Power BI data model (export core tables)
print('\n' + '=' * 80)
print('TASK 21: CREATE POWER BI DATA MODEL - EXPORT TABLES')
print('=' * 80)
import os
import pandas as pd
import numpy as np
out_dir = '../data/processed'
os.makedirs(out_dir, exist_ok=True)
saved = []
# 1) clean_data
if 'df' in globals():
    clean_path = os.path.join(out_dir, 'clean_data.csv')
    df.to_csv(clean_path, index=False, encoding='utf-8')
    saved.append(clean_path)
    print(f'Saved clean_data to: {clean_path}')
else:
    print('Warning: `df` not present in memory; skipping clean_data export.')

# 2) hashtags table (frequency + sentiment if available)
try:
    if 'hashtag_freq_df' in globals():
        hashtags_df = hashtag_freq_df.copy()
    elif 'hashtag_sentiment' in globals():
        hashtags_df = hashtag_sentiment.rename(columns={'hashtag_norm':'hashtag'})
    else:
        # derive from df['hashtags']
        from collections import Counter
        all_hashtags = []
        if 'hashtags' in df.columns:
            for lst in df['hashtags']:
                if isinstance(lst, list):
                    all_hashtags.extend(lst)
        counter = Counter(all_hashtags)
        hashtags_df = pd.DataFrame(list(counter.items()), columns=['hashtag','count']).sort_values('count', ascending=False)
        total = hashtags_df['count'].sum() if len(hashtags_df)>0 else 0
        hashtags_df['percentage'] = (hashtags_df['count'] / total * 100).round(2) if total>0 else 0
    hashtags_path = os.path.join(out_dir, 'hashtags.csv')
    hashtags_df.to_csv(hashtags_path, index=False, encoding='utf-8')
    saved.append(hashtags_path)
    print(f'Saved hashtags table to: {hashtags_path}')
except Exception as e:
    print('Failed to build/save hashtags table:', e)

# 3) sentiment drivers (keyword_drivers) - prefer existing TF-IDF results else compute lightweight version
try:
    kw_path = os.path.join(out_dir, 'keyword_drivers.csv')
    if 'feature_names' in globals() and 'mean_tfidf_pos' in globals() and 'mean_tfidf_neg' in globals():
        import numpy as _np
        kws = list(feature_names)
        pos_scores = _np.round(mean_tfidf_pos, 6).tolist()
        neg_scores = _np.round(mean_tfidf_neg, 6).tolist()
        kd = pd.DataFrame({'keyword': kws, 'tfidf_pos': pos_scores, 'tfidf_neg': neg_scores})
        kd['score_diff'] = (kd['tfidf_pos'] - kd['tfidf_neg']).abs()
        kd = kd.sort_values('score_diff', ascending=False)
        kd.to_csv(kw_path, index=False, encoding='utf-8')
        saved.append(kw_path)
        print(f'Saved keyword drivers to: {kw_path} (from existing TF-IDF)')
    else:
        # lightweight recompute (may be slower) - use cleaned_text or text_no_stopwords
        from sklearn.feature_extraction.text import TfidfVectorizer
        text_candidates = ['cleaned_text','text_no_stopwords','text','tweet','content']
        tcol = next((c for c in text_candidates if c in df.columns), None)
        if tcol is None:
            raise ValueError('No text column for TF-IDF computation')
        pos_texts = df[df.get('sentiment_category','')=='positive'][tcol].fillna('').tolist()
        neg_texts = df[df.get('sentiment_category','')=='negative'][tcol].fillna('').tolist()
        all_texts = pos_texts + neg_texts
        if len(all_texts)==0:
            raise ValueError('No texts available for TF-IDF')
        vec = TfidfVectorizer(max_features=1000, stop_words='english', min_df=2, max_df=0.8)
        X = vec.fit_transform(all_texts)
        fnames = vec.get_feature_names_out()
        import numpy as _np
        X_pos = X[:len(pos_texts)] if len(pos_texts)>0 else _np.zeros((0, X.shape[1]))
        X_neg = X[len(pos_texts):] if len(neg_texts)>0 else _np.zeros((0, X.shape[1]))
        mean_pos = _np.asarray(X_pos.mean(axis=0)).flatten() if X_pos.shape[0]>0 else _np.zeros(X.shape[1])
        mean_neg = _np.asarray(X_neg.mean(axis=0)).flatten() if X_neg.shape[0]>0 else _np.zeros(X.shape[1])
        kd = pd.DataFrame({'keyword': list(fnames), 'tfidf_pos': _np.round(mean_pos,6), 'tfidf_neg': _np.round(mean_neg,6)})
        kd['score_diff'] = (kd['tfidf_pos'] - kd['tfidf_neg']).abs()
        kd = kd.sort_values('score_diff', ascending=False)
        kd.to_csv(kw_path, index=False, encoding='utf-8')
        saved.append(kw_path)
        print(f'Saved keyword drivers to: {kw_path} (recomputed TF-IDF)')
except Exception as e:
    print('Failed to build/save keyword drivers:', e)

# 4) negative analysis bundle: negative_df, negative n-grams, topic summary
try:
    # negative_df
    if 'negative_df' in globals():
        neg = negative_df.copy()
    else:
        if 'polarity' not in df.columns:
            raise ValueError('No polarity column to derive negative_df')
        pos_thr = 0.1; neg_thr = -0.1
        if 'sentiment_category' not in df.columns:
            df['sentiment_category'] = df['polarity'].apply(lambda x: 'positive' if x>pos_thr else ('negative' if x<neg_thr else 'neutral'))
        neg = df[df['sentiment_category']=='negative'].copy()
    neg_path = os.path.join(out_dir, 'negative_df.csv')
    neg.to_csv(neg_path, index=False, encoding='utf-8')
    saved.append(neg_path)
    print(f'Saved negative_df to: {neg_path}')
    # negative n-grams (use existing if present)
    if 'neg_bigram_df' in globals():
        neg_bigram_df.to_csv(os.path.join(out_dir, 'negative_bigrams.csv'), index=False, encoding='utf-8')
        saved.append(os.path.join(out_dir, 'negative_bigrams.csv'))
    if 'neg_trigram_df' in globals():
        neg_trigram_df.to_csv(os.path.join(out_dir, 'negative_trigrams.csv'), index=False, encoding='utf-8')
        saved.append(os.path.join(out_dir, 'negative_trigrams.csv'))
    # topic summary if exists
    topic_sum_path = os.path.join(out_dir, 'negative_topics_summary.csv')
    if os.path.exists(topic_sum_path):
        saved.append(topic_sum_path)
    elif 'topics_df' in globals():
        td = topics_df.copy()
        if 'topic' in td.columns:
            td['cluster_name'] = td['topic'].apply(lambda t: 'cluster_' + str(t))
        td = td.rename(columns={'keywords':'key_terms'})[['cluster_name','key_terms']]
        td.to_csv(topic_sum_path, index=False, encoding='utf-8')
        saved.append(topic_sum_path)
        print(f'Saved negative topic summary to: {topic_sum_path}')
except Exception as e:
    print('Failed to build/save negative analysis artifacts:', e)

print('\nExport summary:')
for p in saved:
    try:
        size_mb = os.path.getsize(p)/(1024*1024)
        print(f'  - {p} ({size_mb:.2f} MB)')
    except Exception:
        print(f'  - {p}')

print('\n' + '=' * 80)
print('✓ Task 21 completed: Power BI tables exported to data/processed')
print('=' * 80)

In [ ]:
# Task 22: Build Overview page - sentiment distribution, time-series trend, word cloud
print('\n' + '=' * 80)
print('TASK 22: OVERVIEW VISUALS (SENTIMENT DISTRIBUTION, TREND, WORDCLOUD)')
print('=' * 80)
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS

out_dir = '../data/processed'
os.makedirs(out_dir, exist_ok=True)

# Ensure df exists
if 'df' not in globals():
    raise ValueError('`df` not loaded in memory. Run Task 1-8 to prepare `df`.')

# 1) Sentiment distribution
if 'sentiment_category' not in df.columns:
    # attempt to compute from polarity column
    if 'polarity' in df.columns:
        pos_thr, neg_thr = 0.1, -0.1
        df['sentiment_category'] = df['polarity'].apply(lambda x: 'positive' if x>pos_thr else ('negative' if x<neg_thr else 'neutral'))
    else:
        raise ValueError('No sentiment information available. Run Task 7 to compute polarity.')

sent_counts = df['sentiment_category'].value_counts().reindex(['positive','neutral','negative']).fillna(0)
plt.figure(figsize=(6,4))
sns.barplot(x=sent_counts.index, y=sent_counts.values, palette=['green','gray','red'])
plt.title('Sentiment Distribution')
plt.ylabel('Count')
plt.xlabel('Sentiment')
sent_img = os.path.join(out_dir, 'overview_sentiment_distribution.png')
plt.tight_layout()
plt.savefig(sent_img, dpi=150)
plt.show()
print(f'Saved sentiment distribution image to: {sent_img}')

# 2) Time-series trend (by date)
# Find a date column
date_candidates = ['date','created_at','timestamp','created','post_date']
date_col = next((c for c in date_candidates if c in df.columns), None)
if date_col is None:
    # try to infer any datetime-like column
    for c in df.columns:
        if 'date' in c.lower() or 'time' in c.lower():
            date_col = c; break

if date_col is None:
    print('No date column found — skipping time-series trend.')
else:
    dt = df[[date_col,'sentiment_category']].copy()
    dt[date_col] = pd.to_datetime(dt[date_col], errors='coerce')
    dt = dt.dropna(subset=[date_col])
    # Resample daily counts per sentiment
    dt.set_index(date_col, inplace=True)
    daily = dt.groupby([pd.Grouper(freq='D'), 'sentiment_category']).size().unstack(fill_value=0)
    # Plot trend (last 90 days if long)
    plot_df = daily.tail(180)
    plt.figure(figsize=(10,4))
    for col in plot_df.columns:
        plt.plot(plot_df.index, plot_df[col], label=col)
    plt.legend()
    plt.title('Sentiment Trend Over Time')
    plt.ylabel('Daily Count')
    plt.xlabel('Date')
    plt.xticks(rotation=45)
    trend_img = os.path.join(out_dir, 'overview_sentiment_trend.png')
    plt.tight_layout()
    plt.savefig(trend_img, dpi=150)
    plt.show()
    print(f'Saved sentiment trend image to: {trend_img}')

# 3) Word cloud (from text_no_stopwords or cleaned_text)
text_col = next((c for c in ['text_no_stopwords','cleaned_text','text','tweet','content'] if c in df.columns), None)
if text_col is None:
    print('No suitable text column for word cloud — skipping.')
else:
    all_text = ' '.join(df[text_col].fillna('').astype(str).tolist())
    if not all_text.strip():
        print('Text column empty — skipping word cloud.')
    else:
        stopwords = set(STOPWORDS)
        wc = WordCloud(width=1200, height=600, background_color='white', stopwords=stopwords, collocations=True).generate(all_text)
        plt.figure(figsize=(12,6))
        plt.imshow(wc, interpolation='bilinear')
        plt.axis('off')
        wc_img = os.path.join(out_dir, 'overview_wordcloud.png')
        plt.tight_layout()
        plt.savefig(wc_img, dpi=150)
        plt.show()
        print(f'Saved word cloud image to: {wc_img}')

print('\n' + '=' * 80)
print('✓ Task 22 completed: Overview visuals generated and saved to data/processed')
print('=' * 80)